In [1]:
# Import necessary libraries

import pandas as pd
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
import os


In [2]:
# Data extraction

try:
    data = pd.read_csv(r'zipco_transaction.csv')
    print('Data Extracted Successfully')
except Exception as e:
    print(f'An error occurred: {e}')


Data Extracted Successfully


In [3]:
data.head()

Date      ProductName  Quantity  UnitPrice StoreLocation  \
0  2023-01-01 00:00:00     Vanilla Cake         2  12.532304         South   
1  2023-01-01 01:00:00  Red Velvet Cake         1   7.083070         South   
2  2023-01-01 02:00:00   Chocolate Cake         5   6.736064         North   
3  2023-01-01 03:00:00      Carrot Cake         2   7.314823         North   
4  2023-01-01 04:00:00  Pizza Pepperoni         1   7.577727          East   

   PaymentType  PromotionApplied Weather  Temperature StaffPerformanceRating  \
0         Cash              True   Rainy    20.654914                   Poor   
1         Cash             False   Rainy    23.549497                Average   
2         Cash              True   Rainy          NaN              Excellent   
3         Cash             False  Cloudy    20.137483                   Poor   
4  Credit Card              True  Cloudy    23.020987                   Good   

   ... DeliveryTime_min     OrderType      CustomerName  \
0  ...               30      In Store     William Adams   
1  ...               33      In Store   Anthony Wiggins   
2  ...               43   Phone Order       Ashley Duke   
3  ...               32  Online Order    Brandon Taylor   
4  ...               58      In Store  Brittany Watkins   

                                     CustomerAddress Customer_PhoneNumber  \
0           9851 David Green\r\nTonyaburgh, VA 02853    (916)427-7276x861   
1      24682 Holly Stravenue\r\nMooreville, NH 13901         769.318.4373   
2  10184 Washington Trace Apt. 679\r\nEast Brandi...         702.520.3286   
3           87194 Jeff Rue\r\nMitchellbury, CA 50463         622-527-9530   
4             850 Julia Groves\r\nHartview, WI 95954         759-517-8359   

                   CustomerEmail        Staff_Name              Staff_Email  \
0             lisa00@example.net      John Bridges    pdavidson@example.com   
1  michellefernandez@example.com     Sarah Bentley     ajohnson@example.net   
2      cooperwilliam@example.com  Connie Cervantes    michele29@example.net   
3             fsilva@example.net   Jessica Stewart      xwilson@example.org   
4       petersjoseph@example.net  Cheryl Carpenter  christine96@example.org   

  DayOfWeek TotalSales  
0    Sunday  25.064608  
1    Sunday   7.083070  
2    Sunday  33.680321  
3    Sunday  14.629647  
4    Sunday   7.577727  

[5 rows x 21 columns]

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1005 non-null   object 
 1   ProductName             1005 non-null   object 
 2   Quantity                1005 non-null   int64  
 3   UnitPrice               1005 non-null   float64
 4   StoreLocation           1005 non-null   object 
 5   PaymentType             1005 non-null   object 
 6   PromotionApplied        1005 non-null   bool   
 7   Weather                 1005 non-null   object 
 8   Temperature             904 non-null    float64
 9   StaffPerformanceRating  1005 non-null   object 
 10  CustomerFeedback        905 non-null    object 
 11  DeliveryTime_min        1005 non-null   int64  
 12  OrderType               1005 non-null   object 
 13  CustomerName            1005 non-null   object 
 14  CustomerAddress         1005 non-null   

In [5]:
# Data Cleaning and Transformation
# Remove Duplicates

data.drop_duplicates(inplace = True)


In [ ]:
# Handle missing numeric values by filling with the mean or median

numeric_columns = data.select_dtypes(include = ['float64', 'int64']).columns
for col in numeric_columns:
    data.fillna({col : data[col].mean()}, inplace = True)

In [ ]:
# Handle missing string values by filling with 'Unknown'

string_columns = data.select_dtypes(include = ['object']).columns
for col in string_columns:
    data.fillna({col : 'Unknown'}, inplace = True)

In [32]:
data['Date'] = pd.to_datetime(data['Date'])

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    1000 non-null   datetime64[ns]
 1   ProductName             1000 non-null   object        
 2   Quantity                1000 non-null   int64         
 3   UnitPrice               1000 non-null   float64       
 4   StoreLocation           1000 non-null   object        
 5   PaymentType             1000 non-null   object        
 6   PromotionApplied        1000 non-null   bool          
 7   Weather                 1000 non-null   object        
 8   Temperature             1000 non-null   float64       
 9   StaffPerformanceRating  1000 non-null   object        
 10  CustomerFeedback        1000 non-null   object        
 11  DeliveryTime_min        1000 non-null   int64         
 12  OrderType               1000 non-null   object        

In [19]:
# Create the product table

products = data[['ProductName']].copy().drop_duplicates().reset_index(drop = True)
products.index.name = 'ProductID'

products = products.reset_index()

In [ ]:
products.head()

(9, 2)

In [ ]:
# Create the customer table

customers = data[['CustomerName', 'CustomerAddress', 'Customer_PhoneNumber', 'CustomerEmail']].copy().drop_duplicates().reset_index(drop = True)
customers.index.name = 'CustomerID'

customers = customers.reset_index()
customers.head()


(1000, 5)

In [ ]:
# Create the staff table

staff = data[['Staff_Name', 'Staff_Email', 'StaffPerformanceRating']].copy().drop_duplicates().reset_index(drop = True)
staff.index.name = 'StaffID'

staff = staff.reset_index()
staff.head()

(1000, 4)

In [27]:
# Create Transactions table

transactions = data.merge(products, on = ['ProductName'], how = 'left')\
    .merge(customers, on = ['CustomerName', 'CustomerAddress', 'Customer_PhoneNumber', 'CustomerEmail'], how = 'left')\
        .merge(staff, on = ['Staff_Name', 'Staff_Email', 'StaffPerformanceRating'], how = 'left')

transactions.columns



Index(['Date', 'ProductName', 'Quantity', 'UnitPrice', 'StoreLocation',
       'PaymentType', 'PromotionApplied', 'Weather', 'Temperature',
       'StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min',
       'OrderType', 'CustomerName', 'CustomerAddress', 'Customer_PhoneNumber',
       'CustomerEmail', 'Staff_Name', 'Staff_Email', 'DayOfWeek', 'TotalSales',
       'ProductID', 'CustomerID', 'StaffID'],
      dtype='object')

In [28]:
transactions = transactions[['Date', 'ProductID', 'UnitPrice', 'Quantity', 'StoreLocation',
       'PaymentType', 'PromotionApplied', 'Weather', 'Temperature', 'CustomerFeedback', 'DeliveryTime_min',
       'OrderType', 'CustomerID', 'StaffID', 'DayOfWeek', 'TotalSales']]

transactions.index.name = 'Transaction_ID'
transactions = transactions.reset_index()

transactions.head()

Transaction_ID                 Date  ProductID  UnitPrice  Quantity  \
0               0  2023-01-01 00:00:00          0  12.532304         2   
1               1  2023-01-01 01:00:00          1   7.083070         1   
2               2  2023-01-01 02:00:00          2   6.736064         5   
3               3  2023-01-01 03:00:00          3   7.314823         2   
4               4  2023-01-01 04:00:00          4   7.577727         1   

  StoreLocation  PaymentType  PromotionApplied Weather  Temperature  \
0         South         Cash              True   Rainy    20.654914   
1         South         Cash             False   Rainy    23.549497   
2         North         Cash              True   Rainy    27.154342   
3         North         Cash             False  Cloudy    20.137483   
4          East  Credit Card              True  Cloudy    23.020987   

  CustomerFeedback  DeliveryTime_min     OrderType  CustomerID  StaffID  \
0          Neutral                30      In Store           0        0   
1         Positive                33      In Store           1        1   
2          Unknown                43   Phone Order           2        2   
3         Positive                32  Online Order           3        3   
4          Neutral                58      In Store           4        4   

  DayOfWeek  TotalSales  
0    Sunday   25.064608  
1    Sunday    7.083070  
2    Sunday   33.680321  
3    Sunday   14.629647  
4    Sunday    7.577727

In [29]:
# convert transaction date column to datetime 

transactions['Date'] = pd.to_datetime(transactions['Date'])

In [31]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction_ID    1000 non-null   int64         
 1   Date              1000 non-null   datetime64[ns]
 2   ProductID         1000 non-null   int64         
 3   UnitPrice         1000 non-null   float64       
 4   Quantity          1000 non-null   int64         
 5   StoreLocation     1000 non-null   object        
 6   PaymentType       1000 non-null   object        
 7   PromotionApplied  1000 non-null   bool          
 8   Weather           1000 non-null   object        
 9   Temperature       1000 non-null   float64       
 10  CustomerFeedback  1000 non-null   object        
 11  DeliveryTime_min  1000 non-null   int64         
 12  OrderType         1000 non-null   object        
 13  CustomerID        1000 non-null   int64         
 14  StaffID           1000 no

In [35]:
# save cleaned data as csv files

data.to_csv('clean_data.csv', index = False)
products.to_csv('products.csv', index = False)
customers.to_csv('customers.csv', index = False)
staff.to_csv('staff.csv', index = False)
transactions.to_csv('transactions.csv', index = False)






In [36]:
# Data Loading
# Load environment variables from the .env files

load_dotenv()

connection_str = os.getenv('AZURE_CONNECTION_STRING_VALUE')
container_name = os.getenv('CONTAINER_NAME')


In [39]:
# Create a BlobServiceClient object
blob_service_object = BlobServiceClient.from_connection_string(connection_str)
container_client = blob_service_object.get_container_client(container_name)

#Load data to Azure Blob Storage
files = [
    (data, 'rawdata/cleaned_zipco_transaction_data.csv'),
    (products, 'cleaneddata/productscsv'),
    (customers, 'cleaneddata/customers.csv'),
    (staff, 'cleaneddata/staff.csv'),
    (transactions, 'cleaneddata/transactions.csv')

]

for file, blob_name in files:
    blob_client = container_client.get_blob_client(blob_name)
    output = file.to_csv(index = False)
    blob_client.upload_blob(output, overwrite = True)
    print(f'{blob_name} loaded into Azure Blob Storage')


rawdata/cleaned_zipco_transaction_data.csv loaded into Azure Blob Storage
cleaneddata/productscsv loaded into Azure Blob Storage
cleaneddata/customers.csv loaded into Azure Blob Storage
cleaneddata/staff.csv loaded into Azure Blob Storage
cleaneddata/transactions.csv loaded into Azure Blob Storage
